In [ ]:
import time
import pickle
import numpy as np
import matplotlib.pyplot as plt
import cv2
import tensorflow as tf
import keras
from keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.models import load_model

In [ ]:
model = load_model('model_mnv2_3cls.h5')

In [ ]:
colors = {0: (0, 255, 0), 1: (0, 255, 255), 2: (0, 0, 255)}
labels = {0: 'Correctly Masked', 1:'Incorrectly Masked', 2: 'Not Masked'}

img_size = (224, 224)

In [ ]:
def classify(img):
    img_array = keras.preprocessing.image.img_to_array(img)
    img_array = preprocess_input(img_array)
    img_array = tf.expand_dims(img_array, 0)
    predictions = model.predict(img_array)
    score = predictions[0]
    class_index = np.argmax(score)
    confidence = round(100*np.max(score), 1)
    return class_index, confidence, score

In [ ]:
cap = cv2.VideoCapture(0)
fps_array=[]
cm_array=[]
im_array=[]
nm_array=[]
run_start_time = time.time()

while cap.isOpened():
    ret, frame = cap.read()  
    start_time = time.time()
    frame_count = 0
    if not ret:
        break
    
 
    frame_count += 1     
    frame = cv2.flip(frame, 1)       
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    img = cv2.resize(rgb, img_size)
    
    try:
        class_index, confidence, score = classify(img)

        text = "{} with {:.1f}% confidence".format(labels[class_index], confidence)
        cv2.rectangle(frame, (0,0), (640,480), 
                      colors[class_index], 10)
        cv2.putText(frame, text, (80, 50), 
                    cv2.FONT_HERSHEY_SIMPLEX, 
                    0.8, colors[class_index], 1)
        
    except Exception as e:
        print(e)
        print("Error!")

    elapsed_time = time.time() - start_time
    fps = round(frame_count/elapsed_time, 1)
    
    cv2.putText(img=frame, text='FPS : ' + str(fps), 
                org=(10, 20), fontFace=cv2.FONT_HERSHEY_SIMPLEX, 
                fontScale=0.5, color=(255,255,255), thickness=1)

    cv2.imshow('Face Mask Detection - MobileNetV2', frame)
    
    fps_array.append(fps) 
    cm_array.append(score[0])
    im_array.append(score[1])
    nm_array.append(score[2])
    
    if cv2.waitKey(1) & 0xFF == 27: #press Esc to exit video 
        break
run_end_time = time.time()

cap.release()
cv2.destroyAllWindows()

total_runtime = run_end_time - run_start_time

In [ ]:
runtime_range = (0, total_runtime+1)
x=np.linspace(0, runtime_range, len(cm_array))

In [ ]:
plt.figure()

plt.plot(x, fps_array, color='darkorchid')

plt.title("Frame Rate for MobileNetV2")
plt.xlabel("Time (s)")
plt.ylim(0, 16.5)
plt.ylabel("Frame Rate (fps)")
plt.grid()
plt.show()

In [ ]:
plt.figure()

plt.plot(x, np.multiply(cm_array, 100), color='green', label='Correctly Masked')
plt.plot(x, np.multiply(im_array, 100), color='gold', label='Incorrectly Masked')
plt.plot(x, np.multiply(nm_array, 100), color='red', label='Not Masked')


handles, labels = plt.gca().get_legend_handles_labels()
by_label = dict(zip(labels, handles))
plt.legend(by_label.values(), by_label.keys(), bbox_to_anchor=(1.05, 0.5), loc='center left')
plt.title("Prediction Confidence for MobileNetV2")
plt.xlabel("Time (s)")
plt.ylabel("Confidence (%)")
plt.grid()
plt.show()